In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [9]:
#First, we need to import the libraries we’ll be using for linear regression
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np
dataset = pd.read_csv('/content/housing.csv')
print("Here are the first five rows of the dataset:")
dataset = dataset.dropna()
Y = dataset['median_house_value']
X = dataset.loc[:,'longitude' : 'median_income']

#spliting it into training and testing portions
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.3,random_state=2003)
x_train_np=x_train.to_numpy()
x_test_np=x_test.to_numpy()
y_train_np=y_train.to_numpy()
y_test_np=y_test.to_numpy()

dataset.head(10)

Here are the first five rows of the dataset:


longitude  latitude  ...  median_house_value  ocean_proximity
0    -122.23     37.88  ...            452600.0         NEAR BAY
1    -122.22     37.86  ...            358500.0         NEAR BAY
2    -122.24     37.85  ...            352100.0         NEAR BAY
3    -122.25     37.85  ...            341300.0         NEAR BAY
4    -122.25     37.85  ...            342200.0         NEAR BAY
5    -122.25     37.85  ...            269700.0         NEAR BAY
6    -122.25     37.84  ...            299200.0         NEAR BAY
7    -122.25     37.84  ...            241400.0         NEAR BAY
8    -122.26     37.84  ...            226700.0         NEAR BAY
9    -122.25     37.84  ...            261100.0         NEAR BAY

[10 rows x 10 columns]

In [0]:
# import the following libraries to build our network 
import torch 
from torch.nn import Conv1d
from torch.nn import MaxPool1d
from torch.nn import Flatten
from torch.nn import Linear
from torch.nn.functional import relu
from torch.utils.data import DataLoader,TensorDataset

In [0]:
 # defining our model and creating  the network
class CnnRegressor(torch.nn.Module):

  def __init__(self, batch_size, inputs, outputs):
    super(CnnRegressor,self).__init__()
    self.batch_size=batch_size
    self.inputs=inputs
    self.outputs=outputs
    self.input_layer=Conv1d(inputs,batch_size,1)
    self.max_pooling_layer=MaxPool1d(1)
    self.conv_layer=Conv1d(batch_size,128,1)
    self.flatten_layer=Flatten()
    self.linear_layer=Linear(128,64)
    self.output_layer=Linear(64,outputs)

  def feed(self,input):

    input=input.reshape((self.batch_size,self.inputs,1))

    output=relu(self.input_layer(input))

    output=self.max_pooling_layer(output)

    output=relu(self.conv_layer(output))
    
    output=self.flatten_layer(output)

    output=self.linear_layer(output)

    output=self.output_layer(output)
    return output

In [12]:
#training the model
from torch.optim import SGD
from torch.nn import L1Loss
!pip install pytorch-ignite
from ignite.contrib.metrics.regression.r2_score import R2Score

In [0]:
batch_size=64
model = CnnRegressor(batch_size, X.shape[1],1)
model.cuda()

In [0]:
def model_loss(model,dataset,train =False,optimizer =None):
  performance = L1Loss()
  score_metric =R2Score()

  avg_loss=0
  avg_score=0
  count=0
  for input,output in iter(dataset):

    predictions=model.feed(input)
    loss=performance(predictions,output)

    score_metric.update([predictions,output])
    score=score_metric.compute()

    if(train):
      optimizer.zero_grad()

      loss.backward()
      optimizer.step()

    avg_loss +=loss.item()
    avg_score +=score
    count += 1
  return avg_loss / count, avg_score / count

In [14]:
epochs=1000

optimizer=SGD(model.parameters(),lr=1e-5)

inputs=torch.from_numpy(x_train_np).cuda().float()
ouputs=torch.from_numpy(y_train_np.reshape(y_train_np.shape[0],1)).cuda().float()

tensor=TensorDataset(inputs,ouputs)
loader=DataLoader(tensor,batch_size,shuffle=True,drop_last=True)

for epoch in range(epochs):

  avg_loss,avg_r2_score=model_loss(model,loader,train=True,optimizer=optimizer)

  print("Epoch"+ str(epoch+1)+ ":\n\tLoss ="+ str(avg_loss)+ "\n\tR^2 Score=" + str (avg_r2_score))

Epoch1:
	Loss =68693.02504904708
	R^2 Score=0.380156382145737
Epoch2:
	Loss =68753.55421454596
	R^2 Score=0.38489853232263394
Epoch3:
	Loss =68477.72913747198
	R^2 Score=0.37963283611452536
Epoch4:
	Loss =69940.86498038117
	R^2 Score=0.3660195102141874
Epoch5:
	Loss =69343.64803461323
	R^2 Score=0.36450251918238535
Epoch6:
	Loss =69109.95978139013
	R^2 Score=0.38800574582060343
Epoch7:
	Loss =68748.50767236546
	R^2 Score=0.3950456545115489
Epoch8:
	Loss =69054.90139784192
	R^2 Score=0.3793551061554404
Epoch9:
	Loss =69221.10063410875
	R^2 Score=0.3748161645973139
Epoch10:
	Loss =69536.04736547085
	R^2 Score=0.37748591538365395
Epoch11:
	Loss =68687.94345571748
	R^2 Score=0.3743400227322414
Epoch12:
	Loss =68728.51438130606
	R^2 Score=0.36950735041974664
Epoch13:
	Loss =68680.56672155269
	R^2 Score=0.3788645248459376
Epoch14:
	Loss =68542.00024523542
	R^2 Score=0.3860929255117346
Epoch15:
	Loss =69218.5545823991
	R^2 Score=0.3534975235534798
Epoch16:
	Loss =70098.26320767937
	R^2 Score=

In [15]:
#testing our model
inputs=torch.from_numpy(x_test_np).cuda().float()
outputs=torch.from_numpy(y_test_np.reshape(y_test_np.shape[0],1)).cuda().float()

tensor=TensorDataset(inputs,outputs)
loader=DataLoader(tensor,batch_size,shuffle=True,drop_last=True)

print(loader)
avg_loss_test,avg_r2_score_test = model_loss(model,loader,train=False,optimizer=None)
print("The model'sL1lossis:"+str(avg_loss_test))
print("The model'sR^2scoreis:"+str(avg_r2_score_test))

The model'sL1lossis:74127.88425164473
The model'sR^2scoreis:0.28512013613073806
